In [2]:
! pip install kagglehub
!pip install implicit
!pip install datasets
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 2.7 MB/s eta 0:00:00a 0:00:01m

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Attempting uninstall: dill
    Found existing installation: dill 0.3.9
    Uninstalling dill-0.3.9:
      Successfully uninstalled dill-0.3.9

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 67.2 MB/s eta 0:00:00

[notice] A new release of pip is avai

In [ ]:
import os
os.kill(os.getpid(), 9)  # This will force a Colab restart


Download and read dataset.
   
It contains 3 tables - 'Users.csv', 'Ratings.csv', 'Books.csv'.

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("arashnic/book-recommendation-dataset")

print("Path to dataset files:", path)

Path to dataset files: /home/jovyan/.cache/kagglehub/datasets/arashnic/book-recommendation-dataset/versions/3


In [4]:
 path = '/home/jovyan/.cache/kagglehub/datasets/arashnic/book-recommendation-dataset/versions/3'

In [5]:
import os

files = os.listdir(path)
print(files)

['DeepRec.png', 'Users.csv', 'Ratings.csv', 'Books.csv', 'classicRec.png', 'recsys_taxonomy2.png']


In [6]:

import pandas as pd

books = pd.read_csv(os.path.join(path, 'Books.csv'))
books.head(5)


/tmp/ipykernel_85/66192168.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv(os.path.join(path, 'Books.csv'))


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [7]:
users = pd.read_csv(os.path.join(path, 'Users.csv'))
users.head(5)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [8]:
ratings = pd.read_csv(os.path.join(path, 'Ratings.csv'))
ratings.head(5)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [29]:
ratings[ratings['ISBN']=='0195153448']

,User-ID,ISBN,Book-Rating,user_index,book_index
9561,2,0195153448,0,850,8967


In [35]:
ratings[ratings['User-ID']==276729]

,User-ID,ISBN,Book-Rating,user_index,book_index
3,276729,052165615X,3,3,3
4,276729,0521795028,6,3,4



This script below processes book descriptions generated in different file from book dataset by cleaning the text, removing stopwords, and lemmatizing words to create a new preprocessed version of the descriptions. The script uses NLTK (Natural Language Toolkit) for text processing and applies the transformations to a Pandas DataFrame.


Steps taken for text processing:

-- convert text to lowercase: This ensures uniform processing and avoids treating "The" and "the" as different words.

-- remove special characters and numbers: Removes everything except letters and spaces (i.e., numbers, punctuation, special symbols).

-- tokenization: Splits text into individual words (tokens).

-- lemmatizer: Used to reduce words to their root form.

-- stop_words: Loads a set of common words that should be removed (e.g., "the", "is", "and", etc.).



In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import pandas as pd


processed_df = pd.read_pickle('books_w_descript.pkl')


# Download necessary NLTK resources
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("stopwords")
nltk.download('punkt_tab')

# Initialize lemmatizer and stop words
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

def preprocess_text(text):
    """Preprocess text: Lowercase, remove special chars, stop words, and lemmatize."""
    if not isinstance(text, str):  # Handle NaN cases
        return ""

    # Lowercase
    text = text.lower()

    # Remove special characters and numbers
    text = re.sub(r"[^a-zA-Z\s]", "", text)

    # Tokenization
    words = word_tokenize(text)

    # Remove stopwords and apply lemmatization
    cleaned_text = " ".join(lemmatizer.lemmatize(word) for word in words if word not in stop_words)

    return cleaned_text


processed_df['Description_vector'] = processed_df['Description'].apply(preprocess_text)

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Code below applies TF-IDF (Term Frequency-Inverse Document Frequency) vectorization to preprocessed book descriptions, transforming text into a numerical representation suitable for machine learning models.
TfidfVectorizer(...) creates a TF-IDF transformation object.

Parameters:
max_features=5000:
Limits the vocabulary to the top 5000 most frequent words (based on TF-IDF scores).
Reduces dimensionality while retaining important features

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Vectorize preprocessed descriptions
tfidf = TfidfVectorizer(stop_words="english", max_features=5000)
tfidf_matrix = tfidf.fit_transform(processed_df['Description_vector'])

# Print shape
print("TF-IDF Matrix Shape:", tfidf_matrix.shape)  # (num_books, num_features)


TF-IDF Matrix Shape: (271355, 5000)


This code below implements a content-based book recommendation system using TF-IDF features and nearest neighbors search. The goal is to find books with similar descriptions to a given book using cosine similarity.

Parameters
-- n_neighbors=6: The model finds the 6 most similar books for each query. 1 book is the query itself, so we return the top 5 recommendations.

-- metric="cosine": Uses cosine similarity to measure book similarity. Cosine similarity is ideal for text data, as it measures the angle between TF-IDF vectors, ignoring magnitude differences.

Function recommend_books_cb takes as input: book_isbn (ISBN of a book), n (number of similar books to return).
If the given ISBN does not exist in the dataset, return an empty list.

Based on the cosine distances to the closest books and the indices of those books in the dataset, it then extracts data for similar books from init dataframe abd returns them.


In [8]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Fit Nearest Neighbors model
nn = NearestNeighbors(n_neighbors=6, metric="cosine", algorithm="auto")  # n_neighbors = 6 (1 target + 5 recommendations)
nn.fit(tfidf_matrix)

def recommend_books_cb(book_isbn, n=5):
    """Find similar books using nearest neighbors (optimized for large datasets)."""
    if book_isbn not in processed_df["ISBN"].values:
        return []

    # Get index of the book
    book_index = processed_df.index[processed_df["ISBN"] == book_isbn][0]

    # Find similar books
    distances, indices = nn.kneighbors(tfidf_matrix[book_index], n_neighbors=n+1)  # +1 to exclude itself
    similar_books = processed_df.iloc[indices[0][0:n+1]]#["ISBN"].tolist()  # Ignore the first result (itself)
    target_book  = processed_df.iloc[indices[0][0]]

    # Return list of dictionaries with ISBN, Title, and Author
    return similar_books[["ISBN", "Book-Title", "Book-Author"]].to_dict(orient="records")


recommendations = recommend_books_cb("0195153448", 5)
for book in recommendations:
    print(f"{book['Book-Title']} by {book['Book-Author']} (ISBN: {book['ISBN']})")


Classical Mythology by Mark P. O. Morford (ISBN: 0195153448)
Mythology by Edith Hamilton (ISBN: 0316341517)
Six Myths of Our Time : Little Angels, Little Monsters, Beautiful Beasts, and More by MARINA WARNER (ISBN: 0679759247)
Who's Who in Non-Classical Mythology by Egerston Sykes (ISBN: 0460861360)
Mythology and You : Classical Mythology and its Relevance in Today's World by Donna Rosenberg (ISBN: 0844255610)
7 Myths About Christianity by Dale Larsen (ISBN: 0830819096)
